This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
GRAPH_NAME = "dollar"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

GRAPH_NAME = "dollar"

GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [4]:
GRAPH_OUTPUT_PATH

WindowsPath('C:/Users/Bobby/Desktop/Fed Challenge 2025/NEUFed2025/figures')

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [5]:
series_codes = {
    "Advanced Foreign Economies": "DTWEXAFEGS",
    "Emerging Market Economies": "DTWEXEMEGS"
}

In [6]:
today = dt.date.today()

quits = fred.get_series(
    series_id=series_codes["Advanced Foreign Economies"],
    observation_start=dt.date(2024, 12, 25),
    observation_end=today
).rename("Advanced Foreign Economies")

# hires = fred.get_series(
#     series_id=series_codes["Emerging Market Economies"],
#     observation_start=dt.date(2024, 12, 25),
#     observation_end=today
# ).rename("Emerging Market Economies")

joined_df = pd.concat(
    [quits],#, hires],
    axis=1,
    join='inner'
).dropna()

joined_df.index.name = "Date"

joined_df.tail()

,Advanced Foreign Economies
Date,
2025-10-10,112.5173
2025-10-14,112.6395
2025-10-15,112.3415
2025-10-16,112.1887
2025-10-17,112.0902


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [10]:
fig = go.Figure()

for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

fig.update_layout(
    title = dict(text = 'Nominal Broad USD Index <br><sup>Daily</sup>'),
    xaxis_title=None,
    yaxis_title=None,
    legend = dict(
        orientation='h',
        y=-0.25
    ),
        height=450,
    width=500,
)

fig.update_yaxes(
    tickformat=".0f",
)

fig.update_xaxes(
    type='date',
    dtick="M4",
    tickformat='%b %Y',
)

ut.add_end_labels(fig = fig)

fig.show()

fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")